# Exercícios - para a parte 02

1- Dar um modelo de camadas crescentes e pedir para eles elaborarem a modelagem.

2- Dar uma curva de um modelo com uma camada muito fina e pedir para modelarem

3- Dar um testemunho de um poço e pedir para modelarem cada camada

**3-** Considere o modelo da tabela abaixo:

|     Camada      |  Resistividade ($\Omega$.m) | Espessura (m) |    Geologia      |
|:---------------:|:---------------------------:|:-------------:|:----------------:|
| 1               | 50.0                        |      0.5      |    Solo          |
| 2               | 300.0                       |      5.0      | Arenito seco     |
| 3               | 35.0                        |      80.0     | Arenito saturado |
| 4               | 1000.0                      |       -       | Granito são      |

**a)** Qual a abertura mínima de eletrodos de injeção de corrente para que se possa identificar de forma confiável a interface superior do granito são? Calcule uma estimativa, justificando-a adequadamente, e demonstre que sua resposta atinge a profundidade desejada fazendo uma modelagem. Considere o arranjo Schlumberger, modificando as aberturas entre os eletrodos A e B no Passo 03 e modificando as aberturas entre os eletrodos M e N no Passo 

**b)** Quantas camadas você observa na curva de resistividade aparente? Por quê?